In [3]:
from tensorflow import keras
import kaggle

In [ ]:
!pip install kaggle

In [63]:
import subprocess
import os
os.makedirs(r"C:\Users\user\Desktop\Sign Language Translator\artifact", exist_ok = True)
command = [
    "kaggle",
    "datasets",
    "download",
    "-d",
    "risangbaskoro/wlasl-processed",
    "-p",
    r"C:\Users\user\Desktop\Sign Language Translator\artifact"
]

subprocess.run(command, check=True)


CompletedProcess(args=['kaggle', 'datasets', 'download', '-d', 'risangbaskoro/wlasl-processed', '-p', 'C:\\Users\\user\\Desktop\\Sign Language Translator\\artifact'], returncode=0)

In [64]:
import zipfile
with zipfile.ZipFile(r"C:\Users\user\Desktop\Sign Language Translator\artifact\wlasl-processed.zip", 'r') as zip_ref:
    zip_ref.extractall(r"C:\Users\user\Desktop\Sign Language Translator\artifact")


In [ ]:
import pathlib
import shutil


def form_dataset():
    os.makedirs(r"C:\Users\user\Desktop\Sign Language Translator\artifact\train", exist_ok = True)
    os.makedirs(r"C:\Users\user\Desktop\Sign Language Translator\artifact\test", exist_ok = True)
    files = [f for f in pathlib.Path(r"C:\Users\user\Desktop\Sign Language Translator\artifact\asl_dataset").iterdir()]
    for folder in files:

       
        images = [f for f in pathlib.Path(folder).iterdir()]
        i = 0
        for image in images:
            if i % 3 == 0:
                shutil.move(image, r"C:\Users\user\Desktop\Sign Language Translator\artifact\test")
            else: 
                shutil.move(image, r"C:\Users\user\Desktop\Sign Language Translator\artifact\train")
            
            i += 1
    

form_dataset()

In [65]:
import os
import shutil
import json

# Paths
json_file_path = r"C:\Users\user\Desktop\Sign Language Translator\artifact\WLASL_v0.3.json"  # Replace with your JSON file
videos_folder = r"C:\Users\user\Desktop\Sign Language Translator\artifact\videos"         # Folder containing videos named by video_id
output_folder = r"C:\Users\user\Desktop\Sign Language Translator\artifact\dataset_split"  # Root folder for train/test/val splits

# Create output directory if not exists
os.makedirs(output_folder, exist_ok=True)

# Load JSON data
with open(json_file_path, "r") as file:
    data = json.load(file)

# Iterate through glosses and instances
for gloss_data in data:
    gloss_name = gloss_data["gloss"]  # Get the gloss/word
    
    for instance in gloss_data["instances"]:
        video_id = instance["video_id"]  # Video file name
        split = instance["split"]        # Split (train, test, val)
        
        # Determine source and destination paths
        source_path = os.path.join(videos_folder, f"{video_id}.mp4")
        dest_folder = os.path.join(output_folder, split, gloss_name)
        dest_path = os.path.join(dest_folder, f"{video_id}.mp4")
        
        # Ensure destination folder exists
        os.makedirs(dest_folder, exist_ok=True)
        
        # Copy video to the destination
        if os.path.exists(source_path):
            shutil.copy(source_path, dest_path)
            print(f"Copied {source_path} to {dest_path}")
        else:
            print(f"Video not found: {source_path}")


Copied C:\Users\user\Desktop\Sign Language Translator\artifact\videos\69241.mp4 to C:\Users\user\Desktop\Sign Language Translator\artifact\dataset_split\train\book\69241.mp4
Video not found: C:\Users\user\Desktop\Sign Language Translator\artifact\videos\65225.mp4
Video not found: C:\Users\user\Desktop\Sign Language Translator\artifact\videos\68011.mp4
Video not found: C:\Users\user\Desktop\Sign Language Translator\artifact\videos\68208.mp4
Video not found: C:\Users\user\Desktop\Sign Language Translator\artifact\videos\68012.mp4
Video not found: C:\Users\user\Desktop\Sign Language Translator\artifact\videos\70212.mp4
Video not found: C:\Users\user\Desktop\Sign Language Translator\artifact\videos\70266.mp4
Video not found: C:\Users\user\Desktop\Sign Language Translator\artifact\videos\07085.mp4
Video not found: C:\Users\user\Desktop\Sign Language Translator\artifact\videos\07086.mp4
Video not found: C:\Users\user\Desktop\Sign Language Translator\artifact\videos\07087.mp4
Copied C:\Users\

In [50]:
base_model = keras.applications.VGG16(
    weights='imagenet',
    input_shape=(224, 224, 3),
    include_top=False)

## Freeze Base Model

In [51]:
# Freeze base model
base_model.trainable = False

## Add Layers to Model

In [53]:
# Create inputs with correct shape
inputs = keras.Input(shape=(224, 224, 3))

x = base_model(inputs, training=False)

# Add pooling layer or flatten layer
x = keras.layers.GlobalAveragePooling2D()(x)

# Add final dense layer
outputs = keras.layers.Dense(6, activation = 'softmax')(x)

# Combine inputs and outputs to create model
model = keras.Model(inputs,outputs)

In [56]:
model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_4 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ vgg16 (Functional)              │ (None, 7, 7, 512)      │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_2      │ (None, 512)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 6)              │         3,078 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,717,766 (56.14 MB)

 Trainable params: 3,078 (12.02 KB)

 Non-trainable params: 14,714,688 (56.13 MB)

## Compile Model

Now it's time to compile the model with loss and metrics options. Remember that we're training on a number of different categories, rather than a binary classification problem.

In [57]:
model.compile(loss='categorical_crossentropy', metrics=['accuracy'])

## Augment the Data

If you'd like, try to augment the data to improve the dataset. Feel free to look at [notebook 04a](04a_asl_augmentation.ipynb) and [notebook 05b](05b_presidential_doggy_door.ipynb) for augmentation examples. There is also documentation for the [Keras ImageDataGenerator class](https://keras.io/api/preprocessing/image/#imagedatagenerator-class). This step is optional, but it may be helpful to get to 92% accuracy.

In [58]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen_train = ImageDataGenerator(    samplewise_center=True,  # set each sample mean to 0
    rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
    zoom_range=0.1,  # Randomly zoom image
    width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=True,  # randomly flip images
    vertical_flip=True,
)
datagen_valid = ImageDataGenerator(samplewise_center=True)

## Load Dataset

Now it's time to load the train and validation datasets. Pick the right folders, as well as the right `target_size` of the images (it needs to match the height and width input of the model you've created). For a reference, check out [notebook 05b](05b_presidential_doggy_door.ipynb).

In [62]:
# load and iterate training dataset
train_it = datagen_train.flow_from_directory(
    r"C:\Users\user\Desktop\Sign Language Translator\artifact\train",
    target_size=(224,224),
    color_mode="rgb",
    class_mode="categorical",
)
# load and iterate validation dataset
valid_it = datagen_valid.flow_from_directory(
    r"C:\Users\user\Desktop\Sign Language Translator\artifact\test",
    target_size=(224,224),
    color_mode="rgb",
    class_mode="categorical",
)

Found 0 images belonging to 36 classes.


## Train the Model

Time to train the model! Pass the `train` and `valid` iterators into the `fit` function, as well as setting the desired number of epochs.

In [8]:
model.fit(train_it,
          validation_data=valid_it,
          steps_per_epoch=train_it.samples/train_it.batch_size,
          validation_steps=valid_it.samples/valid_it.batch_size,
          epochs=9)

Epoch 1/9
37/36 [==============================] - 26s 694ms/step - loss: 2.5725 - accuracy: 0.4239 - val_loss: 1.0127 - val_accuracy: 0.6991
Epoch 2/9
37/36 [==============================] - 17s 470ms/step - loss: 0.7845 - accuracy: 0.7352 - val_loss: 0.4406 - val_accuracy: 0.8693
Epoch 3/9
37/36 [==============================] - 17s 466ms/step - loss: 0.4457 - accuracy: 0.8418 - val_loss: 0.3310 - val_accuracy: 0.8875
Epoch 4/9
37/36 [==============================] - 17s 457ms/step - loss: 0.2950 - accuracy: 0.8942 - val_loss: 0.2288 - val_accuracy: 0.9301
Epoch 5/9
37/36 [==============================] - 17s 457ms/step - loss: 0.2355 - accuracy: 0.9146 - val_loss: 0.1970 - val_accuracy: 0.9392
Epoch 6/9
37/36 [==============================] - 17s 462ms/step - loss: 0.1811 - accuracy: 0.9340 - val_loss: 0.2193 - val_accuracy: 0.9149
Epoch 7/9
37/36 [==============================] - 17s 464ms/step - loss: 0.1280 - accuracy: 0.9442 - val_loss: 0.1507 - val_accuracy: 0.9453
Epoch 

## Unfreeze Model for Fine Tuning

If you have reached 92% validation accuracy already, this next step is optional. If not, we suggest fine tuning the model with a very low learning rate.

In [16]:
# Unfreeze the base model
base_model.trainable = True

# Compile the model with a low learning rate
model.compile(optimizer=keras.optimizers.RMSprop(learning_rate = 0.00001),
              loss = 'categorical_crossentropy' , metrics =['accuracy'])

In [ ]:
model.fit(train_it,
          validation_data=valid_it,
          steps_per_epoch=train_it.samples/train_it.batch_size,
          validation_steps=valid_it.samples/valid_it.batch_size,
          epochs=10 )

Epoch 1/10
37/36 [==============================] - 31s 827ms/step - loss: 0.1083 - accuracy: 0.9611 - val_loss: 0.1446 - val_accuracy: 0.9666
Epoch 2/10
37/36 [==============================] - 19s 507ms/step - loss: 0.0825 - accuracy: 0.9695 - val_loss: 0.1075 - val_accuracy: 0.9726
Epoch 3/10
37/36 [==============================] - 18s 477ms/step - loss: 0.0472 - accuracy: 0.9873 - val_loss: 0.1595 - val_accuracy: 0.9666
Epoch 4/10
37/36 [==============================] - 18s 485ms/step - loss: 0.0353 - accuracy: 0.9873 - val_loss: 0.1698 - val_accuracy: 0.9666
Epoch 5/10
37/36 [==============================] - 18s 487ms/step - loss: 0.0529 - accuracy: 0.9822 - val_loss: 0.0960 - val_accuracy: 0.9757
Epoch 6/10
37/36 [==============================] - 19s 504ms/step - loss: 0.0216 - accuracy: 0.9932 - val_loss: 0.1878 - val_accuracy: 0.9696
Epoch 7/10
37/36 [==============================] - 18s 482ms/step - loss: 0.0274 - accuracy: 0.9924 - val_loss: 0.1136 - val_accuracy: 0.9666

## Evaluate the Model

In [ ]:
model.evaluate(valid_it, steps=valid_it.samples/valid_it.batch_size)